In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

data = pd.read_csv('itamplify_train.csv')

train, test = train_test_split(data, 
                                stratify=data['track_genre'],
                                test_size=0.2,
                                random_state=42)

train, val = train_test_split(train,
                                stratify=train['track_genre'],
                                test_size=0.2,
                                random_state=42)


train_data = TabularDataset(train)
val_data = TabularDataset(val)
test_data = TabularDataset(test)

label = 'track_genre'
predictor = TabularPredictor(label=label).fit(train_data)

In [ ]:
y_pred = predictor.predict(test_data)
y_pred_proba = predictor.predict_proba(test_data)


In [ ]:
predictor.evaluate(test_data)
predictor.leaderboard(test_data)

In [ ]:
predictor.feature_importance(test_data)